In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from itertools import cycle, islice

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mts-ml-summer-school/items.csv
/kaggle/input/mts-ml-summer-school/sample_submission.csv
/kaggle/input/mts-ml-summer-school/users.csv
/kaggle/input/mts-ml-summer-school/interactions.csv


# 📝 Данные 



### `interactions` 

взаимодействия пользователей и книг

- `user_id` - ID пользователя, int64
- `item_id` - ID книги, int64
- `progress` - прогресс по чтению в процентах, int8
- `rating` - рейнтинг книги выставленный пользователем, от 1 до 5, много пропущенных значений
- `start_date` - дата начала чтения книги пользователем

In [2]:
interactions = pd.read_csv('/kaggle/input/mts-ml-summer-school/interactions.csv')

interactions.head().append(interactions.tail())

,user_id,item_id,progress,rating,start_date
0,90133,82910,100,NaN,2018-01-01
1,159130,331068,70,NaN,2018-01-01
2,80061,26540,69,4.0,2018-01-01
3,12811,301895,16,NaN,2018-01-01
4,5778,127872,100,NaN,2018-01-01
1562612,978,321492,50,NaN,2019-12-31
1562613,165713,325032,9,NaN,2019-12-31
1562614,43983,96797,100,5.0,2019-12-31
1562615,63445,157812,5,0.0,2019-12-31
1562616,104572,242176,1,0.0,2019-12-31


In [3]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562617 entries, 0 to 1562616
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   user_id     1562617 non-null  int64  
 1   item_id     1562617 non-null  int64  
 2   progress    1562617 non-null  int64  
 3   rating      323571 non-null   float64
 4   start_date  1562617 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 59.6+ MB


In [4]:
print(f"min interactions date: {interactions['start_date'].min()}")
print(f"max interactions date: {interactions['start_date'].max()}")

min interactions date: 2018-01-01
max interactions date: 2019-12-31


### `items` 

информация о книгах

- `item_id` - ID книги, int64
- `title` - название книги, строка
- `genres` - жанры, строка с разделителем ','
- `authors` - авторы, строка с разделителем ','
- `year` - год публикации, строка (найдите, почему 🤔)


In [5]:
items = pd.read_csv('/kaggle/input/mts-ml-summer-school/items.csv')

items.head().append(items.tail())

,id,title,genres,authors,year
0,248031,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,256084,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,134166,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869
3,281311,Странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831
4,213473,Господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1873
63753,143203,Игрушка палача,"Эротические романы,Любовно-фантастические рома...",Эрика Адамс,2019
63754,27324,Любовь под развесистой клюквой,Современные любовные романы,Маргарита Южина,NaN
63755,359711,"Куда уходят грешницы, или Гробница Наполеона",Современные детективы,Наталья Андреева,NaN
63756,283955,Сварить медведя,"Зарубежные детективы,Исторические детективы,За...",Микаель Ниеми,2017
63757,256666,Подарок моей мечты,Любовное фэнтези,Елизавета Соболянская,2019


In [6]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63758 entries, 0 to 63757
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       63758 non-null  int64 
 1   title    63758 non-null  object
 2   genres   63753 non-null  object
 3   authors  56700 non-null  object
 4   year     49508 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.4+ MB


### `users`

информация о пользователях

- `user_id` - ID пользователя, int64
- `age` - возрастная группа пользователя, строка вида "M_N"
- `sex` - пол пользователя, 1/0

In [7]:
users = pd.read_csv('/kaggle/input/mts-ml-summer-school/users.csv')

users.head().append(users.tail())

,user_id,age,sex
0,0,45_54,1.0
1,1,25_34,0.0
2,2,45_54,0.0
3,3,65_inf,0.0
4,4,18_24,0.0
137249,166060,25_34,0.0
137250,166061,35_44,1.0
137251,166062,45_54,1.0
137252,166065,25_34,0.0
137253,166066,45_54,0.0


In [8]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137254 entries, 0 to 137253
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  137254 non-null  int64  
 1   age      137244 non-null  object 
 2   sex      135640 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.1+ MB


# Пример сабмита

из которого нужно взять id пользователей, для которых будем рекомендовать книги

In [9]:
submission = pd.read_csv('/kaggle/input/mts-ml-summer-school/sample_submission.csv')
submission.head()

,Id,Predicted
0,10001,313 253 245 205 187 168 155 149 146 128 128 12...
1,10002,313 253 245 205 187 168 155 149 146 128 128 12...
2,100152,313 253 245 205 187 168 155 149 146 128 128 12...
3,100197,313 253 245 205 187 168 155 149 146 128 128 12...
4,100284,313 253 245 205 187 168 155 149 146 128 128 12...


In [10]:
print(submission.shape)
print(submission['Id'].nunique())

(3074, 2)
3074


# Бейзлайн № 1 - порекомендуем всем популярные книги

### Класс, который 
- принимает на вход взаимодейтствия 
- выдает список популярных айтемов 
- за указанное количество последних дней (days)

Источник https://www.kaggle.com/sharthz23/metrics-validation-strategies-and-baselines

In [11]:
class PopularRecommender():
    def __init__(self, max_K=100, days=30, item_column='item_id', dt_column='date'):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.dt_column = dt_column
        self.recommendations = []
        
    def fit(self, df):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days)
        self.recommendations = df.loc[df[self.dt_column] > min_date, self.item_column].value_counts().head(self.max_K).index.values
    
    def recommend(self, users=None, N=10):
        recs = self.recommendations[:N]
        if users is None:
            return recs
        else:
            return list(islice(cycle([recs]), len(users)))

In [12]:
# преобразуем даты в datetime формат
interactions['start_date'] = pd.to_datetime(interactions['start_date'])

In [13]:
# обучим модель
pop = PopularRecommender(days=28, dt_column='start_date')
pop.fit(interactions)

In [14]:
# создадим датафрейм с user id из submission
pred_pop = pd.DataFrame({'user_id': submission['Id'].unique()})

# сделаем предикт моделью популярного сразу для всех id
pred_pop['item_id'] = pop.recommend(pred_pop['user_id'], N=10)
pred_pop.head()

,user_id,item_id
0,10001,"[283713, 184549, 276903, 168963, 357309, 14317..."
1,10002,"[283713, 184549, 276903, 168963, 357309, 14317..."
2,100152,"[283713, 184549, 276903, 168963, 357309, 14317..."
3,100197,"[283713, 184549, 276903, 168963, 357309, 14317..."
4,100284,"[283713, 184549, 276903, 168963, 357309, 14317..."


In [15]:
# развернем рекомендации в удобный вертикальный формат и проставим ранк
# ранк = релевантность рекомендации для пользователя
pred_pop = pred_pop.explode('item_id')
pred_pop.head()

,user_id,item_id
0,10001,283713
0,10001,184549
0,10001,276903
0,10001,168963
0,10001,357309


In [16]:
pred_pop['rank'] = pred_pop.groupby('user_id').cumcount() + 1
pred_pop.head()

,user_id,item_id,rank
0,10001,283713,1
0,10001,184549,2
0,10001,276903,3
0,10001,168963,4
0,10001,357309,5


### Функция для конвертации датафрейма с рекомендациями в нужный формат для сабмита

In [17]:
def make_submission(df, id_col='user_id', predicted_col='item_id'):
    
    df_copy = df.copy()
    df_copy.loc[:, id_col] = df_copy[id_col].astype(str)
    df_copy.loc[:, predicted_col] = df_copy[predicted_col].astype(str)
    df_copy = df_copy.groupby(id_col, as_index=False).agg({predicted_col: lambda x: ' '.join(list(x))})
    df_copy.rename(columns={id_col: 'Id', predicted_col: 'Predicted'}, inplace=True)
    
    return df_copy[['Id', 'Predicted']]

 Этот сабмит отражен на лидерборде как `popular_baseline`

In [18]:
submission_pop_90 = make_submission(pred_pop)
submission_pop_90.head()

,Id,Predicted
0,10001,283713 184549 276903 168963 357309 143175 3852...
1,10002,283713 184549 276903 168963 357309 143175 3852...
2,100152,283713 184549 276903 168963 357309 143175 3852...
3,100197,283713 184549 276903 168963 357309 143175 3852...
4,100284,283713 184549 276903 168963 357309 143175 3852...


In [19]:
submission_pop_90['Predicted'][0]

'283713 184549 276903 168963 357309 143175 385281 55466 80003 352049'

In [20]:
submission_pop_90.to_csv('submission_pop_90.csv', index=False)

# 🤓 Пространство для экспериментов:
    
- попробовать разное кол-во дней для сбора популярных книг
- отфильтровать популярное по признакам пользователей. Например: популярное по возрасту или полу
- поэкспериментировать с фильтром по рейтингу    

In [21]:
items_test = items
items_test = items_test.rename(columns = {'id' : 'item_id'})

In [22]:
test_int = pd.merge(interactions, items_test, on = 'item_id')
test_int.head()

,user_id,item_id,progress,rating,start_date,title,genres,authors,year
0,90133,82910,100,NaN,2018-01-01,Ведьма отмщения,Современные детективы,Галина Романова,2013
1,60692,82910,100,NaN,2018-04-11,Ведьма отмщения,Современные детективы,Галина Романова,2013
2,138619,82910,67,NaN,2018-05-24,Ведьма отмщения,Современные детективы,Галина Романова,2013
3,76503,82910,74,NaN,2018-12-21,Ведьма отмщения,Современные детективы,Галина Романова,2013
4,85408,82910,99,NaN,2019-03-06,Ведьма отмщения,Современные детективы,Галина Романова,2013


In [23]:
#самые оцениваемые 
test_int.groupby(['item_id', 'title']).agg({ 'rating' : 'count'})\
 .sort_values(by = 'rating', ascending = False).head(10)

,,rating
item_id,title,
48081,Зулейха открывает глаза,638
205911,Черновик,459
279782,Дневники стюардессы. Лихие 90-е,345
37493,Снеговик,325
112869,"Богатый папа, бедный папа. Роберт Кийосаки (обзор)",324
48356,Мудрые остроты Раневской,320
59445,Чистовик,304
186002,После,301
75731,Земное притяжение,257


In [24]:
interactions['item_id'].value_counts()

80576     6582
319710    2701
178529    2571
260913    2300
79815     2294
          ... 
6573         3
117804       3
372030       3
271527       2
241295       2
Name: item_id, Length: 63758, dtype: int64

In [25]:
#самые оцениваемые с общим рейтом выше среднего ------ оч плохой результат (0.02)
test_int['rating'].mean() # == 4 - средняя оценка по всему датасету
inter_more_than_avg = test_int[test_int['rating'] > 4]
inter_more_than_avg.groupby(['item_id', 'genres']).agg({ 'rating' : 'count'})\
 .sort_values(by = 'rating', ascending = False).head(10)

,,rating
item_id,genres,
48081,Современная русская литература,559
205911,Научная фантастика,355
37493,"Зарубежные детективы,Триллеры,Полицейские детективы,Современные детективы",262
112869,"Личная эффективность,Саморазвитие / личностный рост,Личные финансы",241
59445,Научная фантастика,229
186002,"Зарубежные любовные романы,Современные любовные романы",227
75731,"Современные детективы,Остросюжетные любовные романы",181
330824,"Героическое фэнтези,Книги про волшебников",178
111649,"Зарубежные любовные романы,Современные любовные романы",172


In [26]:
submission_test = submission
submission_test.rename(columns = {'Id' : 'user_id'}, inplace = True)

In [27]:
test_sub = pd.merge(submission_test,users, on = 'user_id', how = 'left')
test_sub.head()

,user_id,Predicted,age,sex
0,10001,313 253 245 205 187 168 155 149 146 128 128 12...,25_34,0.0
1,10002,313 253 245 205 187 168 155 149 146 128 128 12...,NaN,NaN
2,100152,313 253 245 205 187 168 155 149 146 128 128 12...,25_34,0.0
3,100197,313 253 245 205 187 168 155 149 146 128 128 12...,25_34,1.0
4,100284,313 253 245 205 187 168 155 149 146 128 128 12...,45_54,1.0


# Сколько значений неизвестно для тестовой выборки?

In [28]:
test_sub.isna().sum()

user_id        0
Predicted      0
age          547
sex          580
dtype: int64

In [29]:
sub_x = pd.merge(test_sub, interactions)
sub_x = sub_x.drop(['Predicted'], axis = 1)
sub_x.head()

,user_id,age,sex,item_id,progress,rating,start_date
0,10001,25_34,0.0,135899,85,NaN,2018-04-28
1,10001,25_34,0.0,368173,64,NaN,2018-05-08
2,10001,25_34,0.0,78595,81,NaN,2018-11-27
3,10001,25_34,0.0,186002,100,5.0,2018-12-07
4,10001,25_34,0.0,111649,22,NaN,2018-12-19


In [30]:
test_full = pd.merge(test_int, users, on = 'user_id')
test_full.head()

,user_id,item_id,progress,rating,start_date,title,genres,authors,year,age,sex
0,90133,82910,100,NaN,2018-01-01,Ведьма отмщения,Современные детективы,Галина Романова,2013,55_64,0.0
1,90133,26648,100,NaN,2018-10-07,Горький квест. Том 1,Современные детективы,Александра Маринина,2018,55_64,0.0
2,90133,375531,100,NaN,2018-12-08,Понаехавшая,Современная русская литература,Наринэ Абгарян,2011,55_64,0.0
3,90133,248404,99,NaN,2018-01-22,Я орёл. Рассказы и повести,"Книги о приключениях,Юмор и сатира,Современные...",Дмитрий Коробков,NaN,55_64,0.0
4,90133,48081,100,NaN,2018-06-17,Зулейха открывает глаза,Современная русская литература,Гузель Яхина,2015,55_64,0.0


In [31]:
test_int['rating'].mean() # == 4 - средняя оценка по всему датасету
inter_more_than_avg = test_full[test_full['rating'] > 4]
inter_more_than_avg.groupby(['item_id', 'genres', 'age']).agg({ 'rating' : 'count'})\
 .sort_values(by = 'rating', ascending = False).head(10)

rating
item_id genres                                             age          
48081   Современная русская литература                     35_44     158
                                                           25_34     123
                                                           45_54     119
205911  Научная фантастика                                 35_44      87
                                                           25_34      86
                                                           45_54      83
37493   Зарубежные детективы,Триллеры,Полицейские детек... 35_44      71
186002  Зарубежные любовные романы,Современные любовные... 25_34      70
112869  Личная эффективность,Саморазвитие / личностный ... 25_34      67
186002  Зарубежные любовные романы,Современные любовные... 18_24      66

In [32]:
test_full.groupby('item_id').count().rating.sort_values(ascending = False)

item_id
48081     580
205911    420
37493     294
279782    289
48356     280
         ... 
274051      0
274081      0
307636      0
367015      0
196161      0
Name: rating, Length: 63758, dtype: int64

In [33]:
#попробуем найти самое популярное по возрасту

In [34]:
d = {'item_id' : 'count'}
hui = test_full.groupby(['age', 'item_id']).agg(d)
hui = hui.rename_axis(index = {'item_id' : 'book_id'}).sort_values(by = 'item_id', ascending = False)

In [35]:
hui.reset_index(inplace = True)
hui.head()

,age,book_id,item_id
0,25_34,80576,1952
1,35_44,80576,1290
2,18_24,80576,1165
3,45_54,80576,749
4,25_34,249083,676


In [36]:
hui2 = hui.groupby(['age']).head(10)

In [37]:
hui2[hui2['age'] == '65_inf']

,age,book_id,item_id
23,65_inf,178529,401
49,65_inf,319710,331
88,65_inf,270299,246
120,65_inf,79815,212
217,65_inf,260913,154
224,65_inf,3484,152
241,65_inf,37493,146
242,65_inf,210450,146
263,65_inf,386271,139
370,65_inf,340191,111


## Посоветуем самые популярные книги по возрасту там, где указан
### А где не указан - самые популярные за последние 35 дней

In [38]:
#самые популярные книги 18_24
list1824 = '80576 260913 186002 240858 376516 112869 249083 142970 1194 316483'
#самые популярные книги 25_34
list2534 = '80576 249083 48081 112869 112869 37493 170357 240858 25486 23938 3484'
#самые популярные книги 35_44
list3544 = '80576 48081 37493 319710 79815 3484 249083 240858 260913 61100'
#самые популярные книги 45_54
list4554 = '80576 319710 79815 178529 270299 37493 3484 48081 210450 260913'
#самые популярные книги 55_64
list5564 = '178529 319710 79815 270299 80576 210450 3484 37493 260913 386271'
#самые популярные книги 65_inf
list65inf = '178529 319710 270299 79815 260913 3484 37493 210450 386271 340191'

In [39]:
test_sub.loc[test_sub['age'] == '18_24', 'Predicted'] = list1824
test_sub.loc[test_sub['age'] == '25_34', 'Predicted'] = list2534
test_sub.loc[test_sub['age'] == '35_44', 'Predicted'] = list3544
test_sub.loc[test_sub['age'] == '45_54', 'Predicted'] = list4554
test_sub.loc[test_sub['age'] == '55_64', 'Predicted'] = list5564
test_sub.loc[test_sub['age'] == '65_inf', 'Predicted'] = list65inf


In [40]:
test_sub.loc[test_sub['age'].isna(), 'Predicted'] = '283713 184549 276903 168963 357309 143175 385281 80003 55466 267817'

In [41]:
test_sub

,user_id,Predicted,age,sex
0,10001,80576 249083 48081 112869 112869 37493 170357 ...,25_34,0.0
1,10002,283713 184549 276903 168963 357309 143175 3852...,NaN,NaN
2,100152,80576 249083 48081 112869 112869 37493 170357 ...,25_34,0.0
3,100197,80576 249083 48081 112869 112869 37493 170357 ...,25_34,1.0
4,100284,80576 319710 79815 178529 270299 37493 3484 48...,45_54,1.0
...,...,...,...,...
3069,99734,80576 260913 186002 240858 376516 112869 24908...,18_24,0.0
3070,99757,80576 260913 186002 240858 376516 112869 24908...,18_24,0.0
3071,99772,80576 249083 48081 112869 112869 37493 170357 ...,25_34,0.0
3072,99827,80576 48081 37493 319710 79815 3484 249083 240...,35_44,0.0


In [42]:
yolo1 = test_sub.drop(['age', 'sex'], axis = 1)
yolo1 = yolo1.rename(columns = {'user_id' : 'Id'})

In [43]:
yolo1.to_csv('yolo_sub.csv', index=False)

## Попробуем теперь сделать то же самое но для последних 28 дней

In [44]:
test_full.head()

,user_id,item_id,progress,rating,start_date,title,genres,authors,year,age,sex
0,90133,82910,100,NaN,2018-01-01,Ведьма отмщения,Современные детективы,Галина Романова,2013,55_64,0.0
1,90133,26648,100,NaN,2018-10-07,Горький квест. Том 1,Современные детективы,Александра Маринина,2018,55_64,0.0
2,90133,375531,100,NaN,2018-12-08,Понаехавшая,Современная русская литература,Наринэ Абгарян,2011,55_64,0.0
3,90133,248404,99,NaN,2018-01-22,Я орёл. Рассказы и повести,"Книги о приключениях,Юмор и сатира,Современные...",Дмитрий Коробков,NaN,55_64,0.0
4,90133,48081,100,NaN,2018-06-17,Зулейха открывает глаза,Современная русская литература,Гузель Яхина,2015,55_64,0.0


In [45]:
min_date = test_full['start_date'].max().normalize() - pd.DateOffset(days=28)
days28 = test_full[test_full['start_date'] > min_date]
days28.head()

,user_id,item_id,progress,rating,start_date,title,genres,authors,year,age,sex
106,90133,7276,100,5.0,2019-12-22,Помощница палача,Современные детективы,Марина Серова,2020,55_64,0.0
260,90133,313980,99,NaN,2019-12-22,Хирург дьявола,Современные детективы,Марина Серова,2019,55_64,0.0
355,163822,170461,87,NaN,2019-12-22,"Голодание – сильнее таблеток! Исцеление, очище...",Здоровье,Игорь Подопригора,2012,35_44,0.0
360,163822,232743,97,NaN,2019-12-22,Однодневное голодание на воде. Теория. Практик...,"Руководства,Здоровье,Медицина",Анастасия Гагарина,NaN,35_44,0.0
362,163822,9590,5,NaN,2019-12-17,Одна привычка в неделю. Измени себя за год,"Личная эффективность,Саморазвитие / личностный...",Бретт Блюменталь,2015,35_44,0.0


In [46]:
woof = days28.groupby(['age', 'item_id']).agg({'item_id' : 'count'}).rename_axis(index = {'item_id' : 'book_id'}).sort_values(by = 'item_id', ascending = False)
woof = woof.reset_index()

In [47]:
woof = woof.groupby('age').head(10)
woof[woof['age'] == '65_inf'].book_id.values

array([283713, 184549,  55466, 143175, 160349, 374648, 276903,  51423,
       168963,  50718])

In [48]:
#самые популярные книги 18_24
list1824_28 = '80003 283713 184549 385281 168963 56877 276903 334106 287060 267817'
#самые популярные книги 25_34
list2534_28 = '80003 357309 385281 56877 184549 283713 168963 276903 287060 112869'
#самые популярные книги 35_44
list3544_28 = '357309 184549 283713 385281 168963 242176 80003 56877 276903 39077' 
#самые популярные книги 45_54
list4554_28 = '283713 184549 55466 143175 357309 168963 276903 242176 50718 321351'
#самые популярные книги 55_64
list5564_28 = '283713 143175 184549 55466 168963 276903 160349 79499 323949 357309'
#самые популярные книги 65_inf
list65inf_28 = '283713 184549 55466 143175 160349 374648 276903 51423 168963 50718'

#самые популярные в принципе: 
'283713 184549 276903 168963 357309 143175 385281 55466 80003 352049'

'283713 184549 276903 168963 357309 143175 385281 55466 80003 352049'

In [49]:
test_sub.loc[test_sub['age'] == '18_24', 'Predicted'] = list1824_28
test_sub.loc[test_sub['age'] == '25_34', 'Predicted'] = list2534_28
test_sub.loc[test_sub['age'] == '35_44', 'Predicted'] = list3544_28
test_sub.loc[test_sub['age'] == '45_54', 'Predicted'] = list4554_28
test_sub.loc[test_sub['age'] == '55_64', 'Predicted'] = list5564_28
test_sub.loc[test_sub['age'] == '65_inf', 'Predicted'] = list65inf_28

test_sub.loc[test_sub['age'].isna(), 'Predicted'] = '283713 184549 276903 168963 357309 143175 385281 55466 80003 352049'

In [50]:
test_sub

,user_id,Predicted,age,sex
0,10001,80003 357309 385281 56877 184549 283713 168963...,25_34,0.0
1,10002,283713 184549 276903 168963 357309 143175 3852...,NaN,NaN
2,100152,80003 357309 385281 56877 184549 283713 168963...,25_34,0.0
3,100197,80003 357309 385281 56877 184549 283713 168963...,25_34,1.0
4,100284,283713 184549 55466 143175 357309 168963 27690...,45_54,1.0
...,...,...,...,...
3069,99734,80003 283713 184549 385281 168963 56877 276903...,18_24,0.0
3070,99757,80003 283713 184549 385281 168963 56877 276903...,18_24,0.0
3071,99772,80003 357309 385281 56877 184549 283713 168963...,25_34,0.0
3072,99827,357309 184549 283713 385281 168963 242176 8000...,35_44,0.0


In [51]:
yolo2 = test_sub.drop(['age', 'sex'], axis = 1)
yolo2 = yolo2.rename(columns = {'user_id' : 'Id'})
yolo2.to_csv('yolo2.csv', index = False)

# Проделаем то же самое для пола - т.е. укажем самые популярные книги за последние 28 дней сгруппировав по полу.

In [52]:
test_full.head()

,user_id,item_id,progress,rating,start_date,title,genres,authors,year,age,sex
0,90133,82910,100,NaN,2018-01-01,Ведьма отмщения,Современные детективы,Галина Романова,2013,55_64,0.0
1,90133,26648,100,NaN,2018-10-07,Горький квест. Том 1,Современные детективы,Александра Маринина,2018,55_64,0.0
2,90133,375531,100,NaN,2018-12-08,Понаехавшая,Современная русская литература,Наринэ Абгарян,2011,55_64,0.0
3,90133,248404,99,NaN,2018-01-22,Я орёл. Рассказы и повести,"Книги о приключениях,Юмор и сатира,Современные...",Дмитрий Коробков,NaN,55_64,0.0
4,90133,48081,100,NaN,2018-06-17,Зулейха открывает глаза,Современная русская литература,Гузель Яхина,2015,55_64,0.0


In [53]:
sexish = days28.groupby(['sex', 'item_id']).agg({'item_id' : 'count'}).rename_axis(index = {'item_id' : 'book_id'}).sort_values(by = 'item_id', ascending = False)
sexish = sexish.reset_index()

In [54]:
sexish = sexish.groupby('sex').head(10)
sexish[sexish['sex'] == 1.0]['book_id'].values

array([283713, 184549, 276903, 357309,  55466, 385281, 352049, 143175,
       168963, 287060])

In [55]:
#для пола "0"
sex_0_28 = '283713 184549 168963 143175 357309 80003 56877 55466 276903 242176'
#для пола "1"
sex_1_28 = '283713 184549 276903 357309 55466 385281 352049 143175 168963 287060'

#самые популярные в принципе: 
'283713 184549 276903 168963 357309 143175 385281 55466 80003 352049'

'283713 184549 276903 168963 357309 143175 385281 55466 80003 352049'

In [56]:
test_sub.loc[test_sub['sex'] == 1.0, 'Predicted'] = sex_1_28
test_sub.loc[test_sub['sex'] == 0.0, 'Predicted'] = sex_0_28


test_sub.loc[test_sub['sex'].isna(), 'Predicted'] = '283713 184549 276903 168963 357309 143175 385281 55466 80003 352049'

In [57]:
test_sub

,user_id,Predicted,age,sex
0,10001,283713 184549 168963 143175 357309 80003 56877...,25_34,0.0
1,10002,283713 184549 276903 168963 357309 143175 3852...,NaN,NaN
2,100152,283713 184549 168963 143175 357309 80003 56877...,25_34,0.0
3,100197,283713 184549 276903 357309 55466 385281 35204...,25_34,1.0
4,100284,283713 184549 276903 357309 55466 385281 35204...,45_54,1.0
...,...,...,...,...
3069,99734,283713 184549 168963 143175 357309 80003 56877...,18_24,0.0
3070,99757,283713 184549 168963 143175 357309 80003 56877...,18_24,0.0
3071,99772,283713 184549 168963 143175 357309 80003 56877...,25_34,0.0
3072,99827,283713 184549 168963 143175 357309 80003 56877...,35_44,0.0


In [58]:
yolo3 = test_sub.drop(['age', 'sex'], axis = 1)
yolo3 = yolo3.rename(columns = {'user_id' : 'Id'})
yolo3.to_csv('yolo3.csv', index = False)

In [59]:
yolo3

,Id,Predicted
0,10001,283713 184549 168963 143175 357309 80003 56877...
1,10002,283713 184549 276903 168963 357309 143175 3852...
2,100152,283713 184549 168963 143175 357309 80003 56877...
3,100197,283713 184549 276903 357309 55466 385281 35204...
4,100284,283713 184549 276903 357309 55466 385281 35204...
...,...,...
3069,99734,283713 184549 168963 143175 357309 80003 56877...
3070,99757,283713 184549 168963 143175 357309 80003 56877...
3071,99772,283713 184549 168963 143175 357309 80003 56877...
3072,99827,283713 184549 168963 143175 357309 80003 56877...
